In [8]:
!pip install -r requirements.txt

  Using cached alive_progress-2.4.1-py3-none-any.whl (80 kB)
  Using cached grapheme-0.6.0.tar.gz (207 kB)
  Using cached about_time-3.1.1-py3-none-any.whl (9.1 kB)
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210097 sha256=e20d50745cc0807014fb125930147bf35c84a82d8498bbc9f20fc61a9456ca9e
  Stored in directory: /home/theo/.cache/pip/wheels/2d/08/6b/126ea9009f7482fd53a78d0db2ece5aca70af8f4a30445386b
Successfully built grapheme


In [37]:
import json
import pandas as pd
from alive_progress import config_handler, alive_it

config_handler.set_global(force_tty=True)

import ushertools
f = open("tfci.pb", "rb")
f2 = "./hu1.gb"
mat = ushertools.UsherMutationAnnotatedTree(f,f2)

mat.tree.write_tree_newick("/tmp/distance_tree.nwk")

print("Launching chronumental")
import os

os.system(
    "chronumental --tree /tmp/distance_tree.nwk --dates ./tfci.meta.tsv.gz --steps 140 --tree_out /tmp/timetree.nwk --dates_out ./date_comparison.tsv.gz"
)

# %%
import treeswift
print("Reading time tree")
time_tree = treeswift.read_tree("/tmp/timetree.nwk", schema="newick")
time_tree_iter = ushertools.preorder_traversal(time_tree.root)
for i, node in alive_it(enumerate(ushertools.preorder_traversal(mat.tree.root)),
                         title="Adding time tree"):
    time_tree_node = next(time_tree_iter)
    node.time_length = time_tree_node.edge_length
del time_tree
del time_tree_iter


Reading condensed nodes dict |████████████████████████████████████████| 471/471 [100%] in 0.0s (24587.34/s)             
Loading tree
Annotating nuc muts |████████████████████████████████████████| (!) 2849 in 0.0s (114785.08/s) 
Expanding condensed nodes |████████████████████████████████████████| (!) 3604 in 0.1s (38369.65/s) 
Setting branch length |████████████████████████████████████████| (!) 4531 in 0.0s (179422.81/s) 
|████████████████████████████████████████| 4531/4531 [100%] in 0.1s (70589.85/s)                                        
Launching chronumental
Reading time tree
Adding time tree |████████████████████████████████████████| (!) 4531 in 0.0s (170379.34/s) 


In [38]:

def set_x_coords(root):
    """ Set x coordinates for the tree"""
    root.x_dist = 0
    root.x_time = 0
    for node in root.traverse_preorder():
        if node.parent:
            node.x_dist = node.parent.x_dist + node.edge_length
            node.x_time = node.parent.x_time + node.time_length

def set_terminal_y_coords(root):
    for i, node in enumerate(root.traverse_leaves()):
        node.y = i

def set_internal_y_coords(root):
    # Each node should be halfway between the min and max y of its children
    for node in root.traverse_postorder(leaves =False, internal=True):

        child_ys = [child.y for child in node.children]
        node.y = (min(child_ys) + max(child_ys)) / 2
        print(child_ys, node.y)


def get_all_aa_muts(root):
    all_aa_muts = set()
    for node in root.traverse_preorder():
        if node.aa_muts:
            all_aa_muts.update(node.aa_muts)
    return list(all_aa_muts)

def make_aa_object(i, aa_tuple):
    # Tuple format is gene, position, prev, next
    gene, pos, prev, next = aa_tuple
    return {
        "gene":gene,
        "previous_residue": prev,
        "residue_pos": pos,
        "new_residue": next,
        "mutation_id": i,
      }

def get_node_object(node, node_to_index, metadata, aa_mut_tuple_to_index):
    
    object = {}
    object["name"] = node.label if node.label else ""
    object["x_dist"] = node.x_dist
    object["x_time"] = node.x_time
    object["y"] = node.y
    object['mutations'] = [aa_mut_tuple_to_index[aa_tuple] for aa_tuple in node.aa_muts]
    if node.label:
        metadata_row = metadata.loc[node.label]
        for key in metadata_row.index:
            value = metadata_row[key]
            #if value is pd.NaN then set to empty string
            if pd.isna(value):
                value = ""
            object["meta_"+key] = value
    else:
        for key in metadata.columns:
            object["meta_"+key] = ""
    object['parent'] = node_to_index[node.parent] if node.parent else node_to_index[node]
    object['node_id'] = node_to_index[node]
    return object



mat.tree.ladderize(ascending=False)
set_x_coords(mat.tree.root)
set_terminal_y_coords(mat.tree.root)
set_internal_y_coords(mat.tree.root)
nodes_sorted_by_y = sorted(mat.tree.root.traverse_preorder(), key=lambda x: x.y)
all_aa_muts_tuples = get_all_aa_muts(mat.tree.root)
all_aa_muts_objects = [make_aa_object(i, aa_tuple) for i, aa_tuple in enumerate(all_aa_muts_tuples)]
aa_mut_tuple_to_index = {aa_tuple: i for i, aa_tuple in enumerate(all_aa_muts_tuples)}
first_json = {
    "aa_mutations": all_aa_muts_objects,
}
node_to_index = {node: i for i, node in enumerate(nodes_sorted_by_y)}

cols_of_interest = ["strain","genbank_accession", "country", "date", "pangolin_lineage"]

#only load these cols:
metadata = pd.read_csv("./tfci.meta.tsv.gz", sep="\t", usecols=cols_of_interest)
metadata.set_index("strain", inplace=True)

output_file = open("tfci.jsonl", "wt")
output_file.write(json.dumps(first_json) + "\n")
for node in nodes_sorted_by_y:
    node_object = get_node_object(node, node_to_index, metadata, aa_mut_tuple_to_index)
    output_file.write(json.dumps(node_object) + "\n")





In [32]:
metadata.loc['England/TFCI-2707EC6/2020|2020-04-03']

genbank_accession           NaN
date                 2020-04-03
country                 England
pangolin_lineage            B.3
Name: England/TFCI-2707EC6/2020|2020-04-03, dtype: object

In [25]:
row

genbank_accession                NaN
date                      2020-05-11
country                      England
host                             NaN
completeness                     NaN
length                           NaN
Nextstrain_clade                 20B
pangolin_lineage               B.1.1
Nextstrain_clade_usher           20B
pango_lineage_usher            B.1.1
Name: England/TFCI-26F9596/2020|2020-05-11, dtype: object